<a href="https://colab.research.google.com/github/uio-mana/BioData-Advanced-SA/blob/main/Colab_introduction_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Example 1

In [ ]:
# Python Code
def calculate_stats(numbers):
    """
    Calculate the mean and standard deviation of a list of numbers.
    Returns a dictionary with mean and standard deviation.
    """
    from statistics import mean, stdev  # Import required modules
    mean_value = mean(numbers)
    std_dev = stdev(numbers)
    return {"mean": mean_value, "std_dev": std_dev}

# Example usage
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
stats = calculate_stats(numbers)
print(f"Python Result: {stats}")

#Get data from GBIF

In [1]:
# prompt: download and expand dwc archive into occurrence table from https://api.gbif.org/v1/occurrence/download/request/0017682-241107131044228.zip

import requests
import zipfile
import io
import pandas as pd
import os

def download_and_extract_dwca(download_link):
  """Downloads and extracts a Darwin Core Archive (DwCA) from a given URL.

  Args:
    download_link: The URL of the DwCA zip file.

  Returns:
    A pandas DataFrame containing the occurrence data, or None if an error occurs.
  """
  try:
    response = requests.get(download_link, stream=True)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
      for filename in z.namelist():
        if filename.endswith(".csv"):
          occurrence_file = filename
          break
      else:
        # No suitable occurrence file found in the archive
        print("Error: No occurrence.csv file found in the DwCA archive")
        return None

      with z.open(occurrence_file) as f:
          df = pd.read_csv(f, sep='\t')
          return df

  except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")
    return None
  except zipfile.BadZipFile:
    print("Error: Invalid zip file")
    return None
  except KeyError:
      print("Error: No occurrence data found in the archive")
      return None
  except Exception as e:
    print(f"An unexpected error occurred: {e}")
    return None


# Example usage
download_link = "https://api.gbif.org/v1/occurrence/download/request/0017682-241107131044228.zip"
occurrence_df = download_and_extract_dwca(download_link)


# if occurrence_df is not None:
#   print(occurrence_df.head()) # print first 5 rows of the DataFrame
#   # You can now work with the DataFrame (e.g., save it to a file)
#   #occurrence_df.to_csv("occurrence_data.csv", index=False)

In [ ]:
# prompt: make a function to calculate factorial (recursive)

def factorial(n):
  """
  Calculate the factorial of a non-negative integer using recursion.

  Args:
    n: A non-negative integer.

  Returns:
    The factorial of n, or 1 if n is 0.
    Raises ValueError if n is negative.
  """
  if n < 0:
    raise ValueError("Factorial is not defined for negative numbers.")
  elif n == 0:
    return 1
  else:
    return n * factorial(n - 1)

In [3]:
# prompt: show occurrence_df on a static map in jupyter notebooks

import folium
from folium import Figure

# Check if occurrence_df is not None and has the required columns
if occurrence_df is not None and all(col in occurrence_df.columns for col in ['decimalLatitude', 'decimalLongitude']):
    # Create a map centered on the mean latitude and longitude
    m = folium.Map(location=[occurrence_df['decimalLatitude'].mean(), occurrence_df['decimalLongitude'].mean()], zoom_start=5)

    # Add markers for each occurrence
    for index, row in occurrence_df.iterrows():
        try:
            latitude = float(row['decimalLatitude'])
            longitude = float(row['decimalLongitude'])
            folium.Marker([latitude, longitude]).add_to(m)
        except (ValueError, TypeError):
            print(f"Skipping row {index} due to invalid latitude or longitude value.")

    # Display the map
    m
else:
    print("occurrence_df is None or does not contain 'decimalLatitude' and 'decimalLongitude' columns.")
m